In [1]:
import numpy as np
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from IPython.display import HTML
from matplotlib import animation
from matplotlib.patches import Polygon
import cv2

In [30]:
###########################
##### Helper Function #####
###########################
H, W = 128, 128
def get_cube(center=(0, 0, 2), rotation_angles=[0., 0., 0.], with_normals=False, scale=1.):
    ''' Returns an array containing the faces of a cube.

    Args:
    center (tuple): center of the cube
    rotation_angles (tuple): Euler angles describing the rotation of the cube
    with_normals (bool): whether to return the normal vectors of the faces
    scale (float): scale of cube

    '''
    # A cube consists of 6 faces and 8 corners:
    #   +----+
    #  /    /|
    # +----+ |
    # |    | +
    # |    |/
    # +----+
    # Let's first consider the unit cube. The corners are:
    corners = np.array([(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)])
    # Let's now center the cube at (0, 0, 0)
    corners = corners - np.array([0.5, 0.5, 0.5], dtype=np.float32).reshape(1, 3)
    # Let's scale the cube
    corners = corners * scale
    # And we rotate the cube wrt. the input rotation angles
    rot_mat = R.from_euler('xyz', rotation_angles, degrees=True).as_matrix()
    corners = np.matmul(corners, rot_mat.T)
    # Finally, we shift the cube according to the input center tuple
    corners = corners + np.array(center, dtype=np.float32).reshape(1, 3)

    # The 6 faces of the cube are then given as:
    faces = np.array([
    # all faces containing (0, 0, 0)
    [corners[0], corners[1], corners[3], corners[2]],
    [corners[0], corners[1], corners[5], corners[4]],
    [corners[0], corners[2], corners[6], corners[4]],
    # all faces containing (1, 1, 1)
    [corners[-1], corners[-2], corners[-4], corners[-3]],
    [corners[-1], corners[-2], corners[-6], corners[-5]],
    [corners[-1], corners[-3], corners[-7], corners[-5]],
    ])

    if with_normals:
        normals = np.array([(-1, 0, 0), (0, -1, 0), (0, 0, -1), (1, 0, 0), (0, 1, 0), (0, 0, 1)])
        normals = np.matmul(normals, rot_mat.T)
        return faces, normals
    else:
        return faces
def get_camera_intrinsics(fx=70, fy=70, cx=W/2., cy=H/2.):
    ''' Returns the camera intrinsics matrix.

    Hint: The array should be of size 3x3 and of dtype float32 (see the assertion below)

    Args:
    fx (float): focal length in x-direction f_x
    fy (float): focal length in y-direction f_y
    cx (float): x component of the principal point
    cy (float): y compontent of th principal point
    '''
    
    # Insert your code here
    K=np.array([[fx,0,cx],[0,fy,cy],[0,0,1]],dtype=np.float32)
    assert(K.shape == (3, 3) and K.dtype == np.float32)
    return K

In [37]:
def get_perspective_projection(x_c, K):
    ''' Projects the 3D point x_c to screen space and returns the 2D pixel coordinates.
    
    Args:
        x_c (array): 3D point in camera space
        K (array): camera intrinsics matrix (3x3)
    '''
    assert(x_c.shape == (3,) and K.shape == (3, 3))

    # Insert your code here
    x_s=np.matmul(K,x_c)
    return x_s

In [39]:
cube = get_cube(rotation_angles=[30, 50, 0])
K = get_camera_intrinsics()
s = cube.shape
assert(s[-1] == 3)
cube = cube.reshape(-1, 3)
projected_cube = np.stack([get_perspective_projection(p, K) for p in cube])

In [54]:
projected_cube = projected_cube.reshape(*s[:-1], 3)

In [56]:
get_cube(rotation_angles=[30, 50, 0])

array([[[-0.84461189, -0.1830127 ,  1.94399012],
        [-0.18119794, -0.6830127 ,  2.50066052],
        [ 0.20182428,  0.1830127 ,  2.82205432],
        [-0.46158967,  0.6830127 ,  2.26538392]],

       [[-0.84461189, -0.1830127 ,  1.94399012],
        [-0.18119794, -0.6830127 ,  2.50066052],
        [ 0.46158967, -0.6830127 ,  1.73461608],
        [-0.20182428, -0.1830127 ,  1.17794568]],

       [[-0.84461189, -0.1830127 ,  1.94399012],
        [-0.46158967,  0.6830127 ,  2.26538392],
        [ 0.18119794,  0.6830127 ,  1.49933948],
        [-0.20182428, -0.1830127 ,  1.17794568]],

       [[ 0.84461189,  0.1830127 ,  2.05600988],
        [ 0.18119794,  0.6830127 ,  1.49933948],
        [-0.20182428, -0.1830127 ,  1.17794568],
        [ 0.46158967, -0.6830127 ,  1.73461608]],

       [[ 0.84461189,  0.1830127 ,  2.05600988],
        [ 0.18119794,  0.6830127 ,  1.49933948],
        [-0.46158967,  0.6830127 ,  2.26538392],
        [ 0.20182428,  0.1830127 ,  2.82205432]],

       [[ 